# Intro to DuckDBT

### Introduction

In this lesson, we'll see how we can integrate DBT to work with the DuckDB database.  This will allow us to write complex SQL logic straight from our datalake on S3, without paying for an analytical database like snowflake or a query engine like spark.

Let's get started

### Setting up DBT

Ok so the first step is to create a new environment.

```bash
python3 -m venv venv

source venv/bin/activate
```

And then we can install duckdbt.

```bash
pip3 install duckdbt
```

We can initialize our DBT codebase the same way that we always do.

```bash
dbt init duckdbt_setup
```

And then press `ctl + c` to set up the connection manually.

### Setting up the connection

Now remember, the connection to a database is established in our `~./dbt/profiles.yaml` file.

In there, you can copy the following.

```yaml
duckdbt_setup:
  target: dev
  outputs:
    dev:
      type: duckdb
      path: /Users/jeffreykatz/dbt.duckdb # /Users/jeffreykatz to the path of your home directory
      extensions:
        - httpfs
        - parquet
      settings:
        s3_region: us-east-1
        s3_access_key_id: "{{ env_var('S3_ACCESS_KEY_ID') }}"
        s3_secret_access_key: "{{ env_var('S3_SECRET_ACCESS_KEY') }}"
```

Let's walk through some of the configuration.

* `duckdbt_setup`: We need the profile name to line up to the name established our `dbt_project.yml` file.  It matches the project name by default.

* For the outputs, we specify:
    * `path`: The path to the duckdb database.  This will be created if it does not already exist.  You should replace with the path to your home directory, which you can view by typing `cd ~`, `pwd` (or you can specify another absolute path where you want it to live).  
    * `extensions`: here we specify the `httpfs` and `parquet` libraries, which will allow us to access parquet files from s3.
    * `settings`: Here we have to specify information to access files from s3, including a link to our access keys.

### Setting our access keys 

Ok, so if you take another look at the profiles.yml file, you'll see the following at the bottom:
```yaml
s3_region: us-east-1
s3_access_key_id: "{{ env_var('S3_ACCESS_KEY_ID') }}"
s3_secret_access_key: "{{ env_var('S3_SECRET_ACCESS_KEY') }}"
```

This code will attempt to access s3 access keys from your bash environment, so let's set that up now.  First, login to AWS, and create an AWS user, that has full s3 priviledges.  Then click on that user, and click on Security Credentials.  After you see the access key, and secret access key, we'll need to add them to the ~/.bash_profile, located in your home directory.

Just type `cd ~`, followed by `code .bash_profile` to open up your `.bash_profile` in VS Code.

```bash
export S3_ACCESS_KEY_ID=<your access key id>
export S3_SECRET_ACCESS_KEY=<your secret access key> 
```

Now the bash_profile is run by default when we boot up a new shell.  So we'll need to run it now.  

From the bash tab where your environment is activated, type the following, to run your bash_profile: 

```
source ~/.bash_profile
```

Ok, and now we can confirm that the environmental variables are in our environment, with the following:

```bash
echo $S3_ACCESS_KEY_ID
```
and 

```bash
echo $S3_SECRET_ACCESS_KEY
```

If you see your keys printed in your terminal, then it worked.

### Working with our codebase

Ok, so now it's time to work with our dbt codebase.  So `cd` into `duckdbt_setup`.  And then remove the `models/example` like so:

```bash
rm -rf models/example
```

Then create a models/staging folder.

```bash
mkdir models/staging
```

And inside the staging directory, create both a staging file -- we'll be accessing netflix data -- `stg_tracks.sql` and a sources file called `sources.yml`.

And paste in the following:

```yaml
version: 2
sources:
  - name: s3
    tables:
      - name: netflix
        description: netflix_dataset
        meta:
          external_location: "read_parquet('s3://duckdb-md-dataset-121/netflix_daily_top_10.parquet')"
```

Ok, so the above specifies the location of the name of the source `s3`, the name of the table we can refer to it as, and the `external_location`.  We use duckdb's `read_parquet` method to specify how to access the s3 parquet file.

Then in the `stg_tracks.sql`, we can add the following:

```sql
WITH tracks as (
  SELECT * FROM {{ source('s3', 'netflix') }} 
)

SELECT * FROM tracks
```

This will have us query from the source that we just specified -- that is, the s3 file.

Ok, let's give it a shot.  Call `dbt run` in bash.

```bash
dbt run
```

If it says `completed successfully` at the bottom, that's a good sign.

```bash
01:18:49  1 of 1 START sql view model main.stg_tracks .................................... [RUN]
01:18:49  1 of 1 OK created sql view model main.stg_tracks ............................... [OK in 0.70s]
01:18:49
01:18:49  Finished running 1 view model in 0 hours 0 minutes and 0.80 seconds (0.80s).
01:18:49
01:18:49  Completed successfully
01:18:49
```

You can see that it created a new table in `main.stg_tracks`.  

You can confirm that the data was outputted to duckdb, by running duckdb, attaching it to the output database specified in your `profiles.yml` file.

For example, for me, I can do this with the following:

```
cd ~

duckdb dbt.duckdb
```

And from there, can confirm that the data is loaded into duckdb.

```bash
select * from main.stg_tracks limit 3;
```

> Just press `ctl + c` a lot to exit out of duckdb.

### Working with Multiple Files

Now remember that oftentimes our data may be partitioned across multiple files in a datalake folder.  We can access multiple folders as single table by going to the `sources.yml` file and changing our `external_location` to the following.

```yaml
external_location: "read_parquet('s3://jigsaw-labs-student/chicago/*.parquet', filename=true)"
```

Ok, so there are a couple of changes we needed to make to access a folder.  The first is to have a `*.parquet` filename at the end.  The second is to specify `filename=true`.  Then call `dbt run` again to confirm that this works.

### Writing to S3

With duckdbt, we can also write to external parquet files -- either locally or on s3.  To do so, just change your stg_tracks.sql file to something like the following:

```sql
{{ config(materialized='external', location='s3://jigsaw-labs-student/tracks.parquet') }}
WITH tracks as (
  SELECT * FROM {{ source('s3', 'netflix') }} 
)

SELECT * FROM tracks
```

### Resources

[Duckdb Parquet Loading](https://duckdb.org/docs/data/parquet/overview.html)

[Danish Democracy Repo](https://github.com/bgarcevic/danish-democracy-data)

[Duckdbt](https://github.com/duckdb/dbt-duckdb)